<a href="https://colab.research.google.com/github/keisuke0502watanabe/nmr_glue_bruker_data/blob/main/nmr_glue_coffeee_20250529.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nmrglue

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 3.4 MB/s eta 0:00:00


In [2]:
# Google driveのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
#3 ファイルリストの取得
import os
#ドライプから読み込む場合
#データファイルの入ったフォルダパスの指定
# dir_path='/content/drive/MyDrive/Data/koike/nmr/1015-coffeeE/10/pdata/1'
dir_path = '/content/drive/MyDrive/Data/Yokota/nmr/1-Yokota/unidos/10/pdata/1'
# filelist=os.listdir(folder_path)
# for i, file in enumerate(filelist):
#     print(f"{i+1}: {file}")

In [ ]:
#4 指定ファイルからデータ読み込み

# #ファイル番号指定
# num=13
# path=os.path.join(folder_path,filelist[num])
# ##ファイルパス指定してデータ読み込み
# df=pd.read_csv(path, header = None, encoding='utf8', sep=',', names=['chemical shift/ ppm', 'Intensity /a.u.'])
# #読み込みデータ確認
# df.head()

In [6]:
import nmrglue as ng
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

def read_bruker_data(dir_path):
    """
    Read Bruker NMR data and extract key parameters

    Parameters:
    -----------
    dir_path : str or Path
        Path to the Bruker data directory

    Returns:
    --------
    dict
        Dictionary containing the processed data and parameters
    """
    try:
        # Convert string path to Path object for better handling
        path = Path(dir_path)

        # Try reading processed data first, fall back to raw if necessary
        try:
            dic, data = ng.bruker.read(str(path))
        except Exception as e:
            print(f"Could not read processed data: {e}")
            print("Attempting to read raw data...")
            dic, data = ng.bruker.read_raw(str(path))

        # Extract key parameters
        params = {
            'sweep_width': dic['acqus']['SW'],
            'freq': dic['acqus']['SFO1'],
            'offset': dic['acqus']['O1'],
            'nucleus': dic['acqus'].get('NUC1', None),
            'points': dic['acqus'].get('TD', None),
            'experiment': dic['acqus'].get('PULPROG', None)
        }

        # Calculate chemical shift axis
        udic = ng.bruker.guess_udic(dic, data)
        ppm_scale = ng.bruker.make_uc(dic, data, dim=0)
        ppm = ppm_scale.ppm_scale()

        return {
            'dic': dic,
            'data': data,
            'params': params,
            'ppm': ppm
        }

    except Exception as e:
        raise Exception(f"Error reading NMR data: {e}")

def plot_spectrum(nmr_data, title=None):
    """
    Plot the NMR spectrum

    Parameters:
    -----------
    nmr_data : dict
        Dictionary containing NMR data from read_bruker_data
    title : str, optional
        Title for the plot
    """
    plt.figure(figsize=(12, 6))
    plt.plot(nmr_data['ppm'], np.real(nmr_data['data']))
    plt.xlabel('Chemical Shift (ppm)')
    plt.ylabel('Intensity')
    if title:
        plt.title(title)
    plt.grid(True)
    plt.gca().invert_xaxis()  # Conventional NMR format
    plt.show()

# Usage example:
if __name__ == "__main__":
    # Replace with your actual path
    dir_path = "/path/to/nmr/data"

    try:
        nmr_data = read_bruker_data(dir_path)

        # Print parameters
        print("\nNMR Parameters:")
        for key, value in nmr_data['params'].items():
            print(f"{key}: {value}")

        # Plot the spectrum
        plot_spectrum(nmr_data, title=f"NMR Spectrum - {nmr_data['params']['nucleus']}")

    except Exception as e:
        print(f"Error: {e}")

Could not read processed data: directory /path/to/nmr/data does not exist
Attempting to read raw data...
Error: Error reading NMR data: module 'nmrglue.fileio.bruker' has no attribute 'read_raw'


In [7]:
# Additional useful parameters
ns = dic['acqus'].get('NS', None)  # Number of scans
d1 = dic['acqus'].get('D1', None)  # Relaxation delay
te = dic['acqus'].get('TE', None)  # Temperature

In [13]:
import nmrglue as ng
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 処理済みデータのパス
# dir_path = '/content/drive/MyDrive/Data/koike/nmr/1015-coffeeE/10/pdata/1'

try:
    # 処理済みデータの読み込み
    dic, data = ng.bruker.read_pdata(dir_path)

    # ppmスケールの計算
    udic = ng.bruker.guess_udic(dic, data)
    uc = ng.fileiobase.uc_from_udic(udic)
    ppm = uc.ppm_scale()

    # Plotlyでスペクトルを作成
    fig = go.Figure()

    # スペクトルのトレース追加
    fig.add_trace(
        go.Scatter(
            x=ppm,
            y=np.real(data),
            mode='lines',
            name='NMR Spectrum',
            line=dict(color='blue', width=1)
        )
    )

    # レイアウトの設定
    fig.update_layout(
        title='1H NMR Spectrum',
        xaxis=dict(
            title='Chemical Shift (ppm)',
            autorange='reversed',  # ppmスケールを反転
            gridcolor='lightgray',
            showgrid=True
        ),
        yaxis=dict(
            title='Intensity',
            gridcolor='lightgray',
            showgrid=True
        ),
        plot_bgcolor='white',  # プロット背景色
        width=900,  # 図の幅
        height=500,  # 図の高さ
        showlegend=False
    )

    # 図の表示
    fig.show()

    # パラメータの表示
    print("\nNMRパラメータ:")
    print(f"観測周波数: {dic['acqus']['SFO1']} MHz")
    print(f"スペクトル幅: {dic['acqus']['SW']} Hz")
    print(f"データポイント数: {dic['acqus']['TD']}")

except Exception as e:
    print(f"エラー: {e}")


NMRパラメータ:
観測周波数: 400.132470966543 MHz
スペクトル幅: 20.0254193154209 Hz
データポイント数: 65536
